<a href="https://colab.research.google.com/github/maytlim/doh_data/blob/main/vacancies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl -O https://raw.githubusercontent.com/maytlim/doh_data/main/landline.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2688  100  2688    0     0   6892      0 --:--:-- --:--:-- --:--:--  6892


In [2]:
import requests
import pandas as pd
import datetime
import landline
import gdown

In [32]:
  pd.options.display.float_format = '{:,.0f}'.format

In [4]:
try:
  import pdfx
except ModuleNotFoundError:
  !pip install pdfx
  import pdfx

In [5]:
#Code from https://gist.github.com/korakot/51a917e1f53891d53be223439b0f75c1
from google.colab import auth
auth.authenticate_user()  # must authenticate

'''list all ids of files directly under folder folder_id'''
def folder_list(folder_id):
  from googleapiclient.discovery import build
  gdrive = build('drive', 'v3', cache_discovery=False).files()
  #gdrive = build('drive', 'v3').files()
  res = gdrive.list(q="'%s' in parents" % folder_id).execute()
  return [f['id'] for f in res['files']]

'''download all files from a gdrive folder to current directory'''
def folder_download(folder_id):
  for fid in folder_list(folder_id):
    !gdown -q --id $fid

In [6]:
ddrop_readme = requests.get('http://bit.ly/DataDropPH').url
ddrop_readme = ddrop_readme.split('/')[-1].split('?')[0]
#print(ddrop_readme)
folder_download(ddrop_readme)

In [7]:
readme_pdf = !ls *.pdf
readme_pdf = readme_pdf[0][1:-1]
readme_pdf
pdfx_tmp = pdfx.PDFx(readme_pdf)
ddlinks = pdfx_tmp.get_references_as_dict()

In [8]:
month_day = readme_pdf.split('_')
month = int(month_day[0][-2:])
day = int(month_day[1][:2])
month, day
year = 2021
print('DOH DataDrop Release:', year, month, day)

DOH DataDrop Release: 2021 4 1


In [9]:
downloaded = False
for ddlink in ddlinks['url']:
  if downloaded == False:
    if ddlink[:15] == 'https://bit.ly/' and ddlink[-3:] != 'ive':
      datadrop = requests.get(ddlink).url
      datadrop = datadrop.split('/')[-1].split('?')[0]
      folder_download(datadrop)
      print('Downloading Data Drop...')
      downloaded = True
  else:
    continue

In [10]:
data_date = datetime.datetime(year, month, day)
#data_dir ='DOH COVID Data Drop_'  + data_date.strftime('%Y%m%d') + '/'

In [11]:
landline_masterlist = landline.make_masterlist()

In [12]:
# hospital daily patient census
cap_filename = 'DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') +' - 05 DOH Data Collect - Daily Report.csv'
# hospital weekly inventory of supplies
sup_filename = 'DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') +' - 06 DOH Data Collect - Weekly Report.csv'
print('weekly supplies inventory:', sup_filename)
print('daily capability count:', cap_filename)
hosp_code = 'hfhudcode' 
#hosp_name = 'cfname'

weekly supplies inventory: DOH COVID Data Drop_ 20210401 - 06 DOH Data Collect - Weekly Report.csv
daily capability count: DOH COVID Data Drop_ 20210401 - 05 DOH Data Collect - Daily Report.csv


In [13]:
#load capability data
df_cap = pd.read_csv(cap_filename)
df_cap['cap_reportdate'] = pd.to_datetime(df_cap['reportdate'])
df_cap.drop(['updateddate', 'addeddate', 'reportdate', 'id', 'north_coord', 'east_coord'], axis=1, inplace=True)
df_cap[hosp_code] = df_cap[hosp_code].apply(lambda x: x.strip())

In [14]:
!gdown 'https://raw.githubusercontent.com/maytlim/doh_data/main/nhfr.json'

Downloading...
From: https://raw.githubusercontent.com/maytlim/doh_data/main/nhfr.json
To: /content/nhfr.json
19.6MB [00:00, 91.8MB/s]


In [15]:
df_nhfr = pd.read_json('nhfr.json')
print('Refer to https://nhfr.doh.gov.ph/ for the latest data')

Refer to https://nhfr.doh.gov.ph/ for the latest data


In [16]:
df_cap = df_cap.merge(how='left', right=df_nhfr, left_on='hfhudcode', right_on='Health Facility Code')

In [17]:
#load supplies data
df_sup = pd.read_csv(sup_filename)
df_sup['sup_reportdate'] = pd.to_datetime(df_sup['reportdate'])
df_sup.drop(['updateddate', 'addeddate', 'reportdate', 'id'], axis=1, inplace=True)
df_sup[hosp_code] = df_sup[hosp_code].apply(lambda x: x.strip())

In [18]:
# quarantine facility daily patient census
quarantine_daily = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 08 Quarantine Facility Data - Daily Report.csv')
# quarantine facility weekly inventory of supplies
quarantine_weekly = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 09 Quarantine Facility Data - Weekly Report.csv')

In [19]:
# HCW availability
#baseline_v3 = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 10 DOH Data Collect v3 - Baseline.csv')
# mechvent, swabbing, ipc
#baseline_v4 = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 11 DOH Data Collect v4 - Baseline.csv')
# quarantine and isolation facilities
#ttmf = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 12 DDC TTMF Tracker v1 - Baseline.csv')

In [20]:
#print('{0}\n{1}'.format(df_cap.keys(), df_sup.keys()))
#print('{0}\n{1}'.format(quarantine_weekly.keys(), quarantine_daily.keys()))

In [21]:
reg_lookup = {'NCR': 'NATIONAL CAPITAL REGION (NCR)', 'ARMM': 'AUTONOMOUS REGION IN MUSLIM MINDANAO (ARMM)', 
              'CAR': 'CORDILLERA ADMINISTRA TIVE REGION (CAR)', 'I': 'REGION I (ILOCOS REGION)', 
              'II': 'REGION II (CAGAYAN VALLEY)', 'III': 'REGION III (CENTRAL LUZON)', 
              'IV-A': 'REGION IV-A (CALABAR ZON)', 'IV-B': 'REGION IV-B (MIMAROPA)', 
              'V': 'REGION V (BICOL REGION)', 'VI': 'REGION VI (WESTERN VISAYAS)', 
              'VII': 'REGION VII (CENTRAL VISAYAS)', 'VIII': 'REGION VIII (EASTERN VISAYAS)', 
              'IX': 'REGION IX (ZAMBOANGA PENINSULA)', 'X': 'REGION X (NORTHERN MINDANAO)', 
              'XI': 'REGION XI (DAVAO REGION)', 'XII': 'REGION XII (SOCCSKSA RGEN)', 
              'XIII': 'REGION XIII (CARAGA)'}

In [22]:
prefix = ['ics', 'resdoc', 'nurse', 'medtech', 'respthe', 'radtech', 'support']
suffix = ['_total', '_er', '_icu', '_ward'] 
triage = ['resdoc_triage', 'nurse_triage', 'support_triage']
added = ['Augmen_total']

In [23]:
def day_ago_from(n, from_date):
  showdate = from_date - datetime.timedelta(hours=24)*n
  return showdate.strftime('%Y-%m-%d')

In [24]:
def show_filtered(df_cap, days_ago, region='NATIONAL CAPITAL REGION (NCR)'):
  filter = (df_cap.mechvent_v > 0) & \
    (df_cap.cap_reportdate == day_ago_from(days_ago, data_date)) & \
    (df_cap.region == region) & \
    (df_cap.icu_v > 0) & \
    (df_cap.icu_o > 0)
  df_vacant = df_cap[filter][['cap_reportdate', 'cfname', 'city_mun', 'icu_v', 'isolbed_v', 'mechvent_v', 'beds_ward_v', 
                            'icu_o', 'isolbed_o', 'mechvent_o', 'beds_ward_o', 
                            'Landline Number', 'Landline Number 2', 'Fax Number', 'Ownership Major Classification']]
  df_vacant.rename({'Landline Number': 'landline', 'Landline Number 2': 'landline2', 'Fax Number': 'fax', 'Ownership Major Classification':'ownership'}, axis=1, inplace=True)
  df_vacant['landline'] = df_vacant['landline'].apply(lambda x: landline.convert_landline(x, landline_masterlist))
  df_vacant['landline2'] = df_vacant['landline2'].apply(lambda x: landline.convert_landline(x, landline_masterlist))
  df_vacant['fax'] = df_vacant['fax'].apply(lambda x: landline.convert_landline(x, landline_masterlist))
  df_vacant.sort_values(by='icu_v', ascending=False, inplace=True)
  df_vacant.reset_index(inplace=True)
  return df_vacant

In [25]:
show_filtered(df_cap, days_ago=3)

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership
0,199428,2021-03-29,DR. JOSE N. RODRIGUEZ MEMORIAL HOSPITAL,CALOOCAN CITY,87,172,59,0,13,328,1,0,(02) 8294 2571,(02) 8962 9885,(02) 8962 8209,Government
1,40226,2021-03-29,JUSTICE JOSE ABAD SANTOS GENERAL HOSPITAL,BINONDO,8,11,5,2,4,20,1,0,(02) 8254 2453,(02) 5310 1984,None,Government
2,227673,2021-03-29,PASIG CITY GENERAL HOSPITAL,CITY OF PASIG,8,17,10,0,12,9,5,0,(02) 8273 7981,(02) 8643 3333,(02) 8642 7380,Government
3,11424,2021-03-29,ST. LUKE?S MEDICAL CENTER - GLOBAL CITY,TAGUIG CITY,6,1,9,4,16,6,10,102,(02) 8789 7700,None,None,Private
4,109124,2021-03-29,DR. VICTOR R. POTENCIANO MEDICAL CENTER,CITY OF MANDALUYONG,5,9,4,0,3,6,4,27,(02) 8464 9999,None,(02) 8532 1764,Private
5,102317,2021-03-29,VALENZUELA MEDICAL CENTER,CITY OF VALENZUELA,4,2,8,37,3,0,0,16,(02) 8294 6711,None,(02) 8291 4259,Government
6,5985,2021-03-29,UNIVERSITY OF THE EAST RAMON MAGSAYSAY MEMORIA...,QUEZON CITY,4,2,1,2,3,10,11,14,(02) 8716 1853,None,(02) 8716 1853,Private
7,321933,2021-03-29,RIZAL MEDICAL CENTER,CITY OF PASIG,3,32,17,0,14,68,5,0,(02) 8865 8400,(02) 8671 9616,(02) 8671 9616,Government
8,16874,2021-03-29,PHILIPPINE HEART CENTER,QUEZON CITY,3,65,2,0,10,64,5,0,(02) 8925 2401,(02) 8922 0551,(02) 8922 0551,Government
9,272261,2021-03-29,UNIVERSITY OF PERPETUAL HELP DALTA MEDICAL CEN...,CITY OF LAS PIÑAS,3,3,9,0,11,33,9,0,(02) 8874 8515,(02) 8873 7210,(02) 8873 7210,Private


In [26]:
show_filtered(df_cap, days_ago=2)

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership
0,199429,2021-03-30,DR. JOSE N. RODRIGUEZ MEMORIAL HOSPITAL,CALOOCAN CITY,69,86,36,0,31,314,24,0,(02) 8294 2571,(02) 8962 9885,(02) 8962 8209,Government
1,227674,2021-03-30,PASIG CITY GENERAL HOSPITAL,CITY OF PASIG,10,26,10,0,10,10,5,0,(02) 8273 7981,(02) 8643 3333,(02) 8642 7380,Government
2,40227,2021-03-30,JUSTICE JOSE ABAD SANTOS GENERAL HOSPITAL,BINONDO,7,8,5,0,5,23,1,2,(02) 8254 2453,(02) 5310 1984,None,Government
3,5986,2021-03-30,UNIVERSITY OF THE EAST RAMON MAGSAYSAY MEMORIA...,QUEZON CITY,6,2,1,1,4,9,10,15,(02) 8716 1853,None,(02) 8716 1853,Private
4,324654,2021-03-30,PASAY CITY GENERAL HOSPITAL,PASAY CITY,6,7,1,0,6,11,3,28,(02) 8833 6022,(02) 8831 3285,(02) 8551 3735,Government
5,109125,2021-03-30,DR. VICTOR R. POTENCIANO MEDICAL CENTER,CITY OF MANDALUYONG,5,4,4,0,3,5,4,30,(02) 8464 9999,None,(02) 8532 1764,Private
6,11425,2021-03-30,ST. LUKE?S MEDICAL CENTER - GLOBAL CITY,TAGUIG CITY,5,2,4,7,17,5,15,99,(02) 8789 7700,None,None,Private
7,119992,2021-03-30,QUEZON CITY GENERAL HOSPITAL,QUEZON CITY,4,9,4,0,25,72,0,0,(02) 8426 1383,(02) 8426 1321,(02) 8920 7081,Government
8,27200,2021-03-30,NATIONAL CHILDREN?S HOSPITAL,QUEZON CITY,3,49,5,0,10,28,10,0,(02) 8724 0656,None,(02) 8721 9139,Government
9,65199,2021-03-30,JOSE R. REYES MEMORIAL MEDICAL CENTER,SANTA CRUZ,3,15,4,19,4,50,3,44,(02) 8711 9491,(02) 8711 9498,(02) 8732 1077,Government


In [27]:
show_filtered(df_cap, days_ago=1)

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership
0,199430,2021-03-31,DR. JOSE N. RODRIGUEZ MEMORIAL HOSPITAL,CALOOCAN CITY,65,85,35,0,35,315,25,0,(02) 8294 2571,(02) 8962 9885,(02) 8962 8209,Government
1,227675,2021-03-31,PASIG CITY GENERAL HOSPITAL,CITY OF PASIG,8,26,10,0,12,12,5,0,(02) 8273 7981,(02) 8643 3333,(02) 8642 7380,Government
2,109126,2021-03-31,DR. VICTOR R. POTENCIANO MEDICAL CENTER,CITY OF MANDALUYONG,7,15,4,0,2,9,5,31,(02) 8464 9999,None,(02) 8532 1764,Private
3,40228,2021-03-31,JUSTICE JOSE ABAD SANTOS GENERAL HOSPITAL,BINONDO,7,3,5,0,5,28,1,2,(02) 8254 2453,(02) 5310 1984,None,Government
4,221409,2021-03-31,TONDO MEDICAL CENTER,TONDO I / II,6,22,1,0,12,22,5,82,(02) 8522 1174,(02) 8522 9244,(02) 8252 8661,Government
5,5987,2021-03-31,UNIVERSITY OF THE EAST RAMON MAGSAYSAY MEMORIA...,QUEZON CITY,5,2,1,1,5,9,10,15,(02) 8716 1853,None,(02) 8716 1853,Private
6,119993,2021-03-31,QUEZON CITY GENERAL HOSPITAL,QUEZON CITY,5,10,1,0,21,71,3,0,(02) 8426 1383,(02) 8426 1321,(02) 8920 7081,Government
7,183963,2021-03-31,DE LOS SANTOS MEDICAL CENTER,QUEZON CITY,3,7,3,4,5,17,5,6,(02) 8723 0041,(02) 8893 5762,None,Private
8,11426,2021-03-31,ST. LUKE?S MEDICAL CENTER - GLOBAL CITY,TAGUIG CITY,3,2,7,3,19,5,12,103,(02) 8789 7700,None,None,Private
9,16876,2021-03-31,PHILIPPINE HEART CENTER,QUEZON CITY,3,63,2,0,10,65,5,0,(02) 8925 2401,(02) 8922 0551,(02) 8922 0551,Government


In [28]:
show_filtered(df_cap, days_ago=1, region=reg_lookup['II'])

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership
0,103137,2021-03-31,REGION II TRAUMA AND MEDICAL CENTER (VETE...,BAYOMBONG (CAPITAL),12,22,6,0,5,26,5,27,+63 783212090,+63 788053560,None,Government


In [29]:
show_filtered(df_cap, days_ago=1, region=reg_lookup['IV-A'])

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership
0,187760,2021-03-31,GENERAL EMILIO AGUINALDO MEMORIAL HOSPITAL,TRECE MARTIRES CITY (CAPITAL),4,17,10,12,2,17,0,10,+63 464190063,None,None,Government
1,332823,2021-03-31,DE LA SALLE UNIVERSITY MEDICAL CENTER,CITY OF DASMARIÑAS,4,10,13,0,4,34,3,0,(02) 8988 3100,+63 464818000,None,Private
2,26383,2021-03-31,BATANGAS MEDICAL CENTER,BATANGAS CITY (CAPITAL),2,21,27,0,9,18,3,28,+63 437408307,None,+63 437230165,Government
3,55398,2021-03-31,"LOS BA?OS DOCTORS HOSPITAL AND MEDICAL CENTER,...",LOS BAÑOS,2,7,1,0,3,9,1,0,+63 495360100,None,+63 495361225,Private
4,57851,2021-03-31,MEDICAL CENTER WESTERN BATANGAS,BALAYAN,2,3,1,3,1,2,1,1,+63 434071103,None,+63 434071101,Private
5,324110,2021-03-31,DANIEL O. MERCADO MEDICAL CENTER,CITY OF TANAUAN,2,7,2,0,2,3,0,7,+63 437781810,+63 437780960,None,Private
6,30182,2021-03-31,UNIHEALTH-SOUTHWOODS HOSPITAL AND MEDICAL CENT...,CITY OF BIÑAN,1,19,15,7,4,2,0,8,+63 498616385,None,None,Private
7,37789,2021-03-31,HEALTHSERV LOS BA?OS MEDICAL CENTER,LOS BAÑOS,1,4,1,0,3,8,0,9,+63 495364858,None,+63 495844695,Private
8,86017,2021-03-31,EMILIO AGUINALDO COLLEGE MEDICAL CENTER CAVITE,CITY OF DASMARIÑAS,1,4,4,3,2,1,2,4,+63 464163010,None,+63 465194210,Private
9,345069,2021-03-31,LAGUNA PROVINCIAL HOSPITAL-SAN PABLO CITY DIST...,SAN PABLO CITY,1,8,3,0,7,30,4,0,+63 495211677,+63 495620311,+63 495620371,Government


In [30]:
show_filtered(df_cap, days_ago=1, region=reg_lookup['III'])

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership
0,265470,2021-03-31,JOSE B. LINGAD MEMORIAL REGIONAL HOSPITAL,CITY OF SAN FERNANDO (CAPITAL),17,0,18,38,65,0,10,217,+63 459613544,None,None,Government
1,143531,2021-03-31,DR. PAULINO J. GARCIA MEMORIAL RESEARCH & MEDI...,CABANATUAN CITY,10,38,14,22,6,10,2,34,+63 444638286,+63 444639937,None,Government
2,203517,2021-03-31,BULACAN MEDICAL CENTER,CITY OF MALOLOS (CAPITAL),10,74,49,28,14,22,1,72,+63 447910630,None,None,Government
3,239915,2021-03-31,BATAAN GENERAL HOSPITAL AND MEDICAL CENTER,CITY OF BALANGA (CAPITAL),9,29,19,6,30,70,29,6,+63 472372269,None,None,Government
4,97697,2021-03-31,RAMOS GENERAL HOSPITAL,CITY OF TARLAC (CAPITAL),4,6,2,0,1,10,0,0,+63 459827074,None,None,Private
5,270089,2021-03-31,"GRACE GENERAL HOSPITAL, INC.",CITY OF SAN JOSE DEL MONTE,3,3,1,0,1,11,0,0,+63 445194838,None,None,Private
6,102592,2021-03-31,JAMES L. GORDON MEMORIAL HOSPITAL,OLONGAPO CITY,2,40,3,3,2,29,2,5,+63 476021229,+63 476021230,+63 476021228,Government
7,171445,2021-03-31,PRES. RAMON MAGSAYSAY MEMORIAL HOSPITAL,IBA (CAPITAL),2,15,3,27,6,22,1,153,+63 478117212,None,None,Government
8,151904,2021-03-31,"SE?OR STO. NI?O HOSPITAL, INC.",CAMILING,1,1,4,2,3,6,0,2,+63 459340806,None,None,Private
9,233725,2021-03-31,ISAAC AND CATALINA MEDICAL CENTER,CITY OF BALANGA (CAPITAL),1,18,2,0,1,0,0,0,+63 472371435,None,None,Private


In [31]:
roi = reg_lookup['NCR']
#roi = reg_lookup['II']
#roi = reg_lookup['IV-A']
filter = (df_cap.cap_reportdate == day_ago_from(1, data_date)) & (df_cap.region == roi)
occupied = df_cap[filter]['icu_o'].sum()
vacant = df_cap[filter]['icu_v'].sum()
pct_used = (occupied / (vacant + occupied)) * 100
print('[', roi, ']:', round(pct_used), '% ICU rooms utilized, ',  int(vacant), 'vacant')

[ NATIONAL CAPITAL REGION (NCR) ]: 76 % ICU rooms utilized,  190 vacant
